In [61]:
# 必要なライブラリのインストール
# pip install transformers fugashi ipadic demoji neologdn unidic-lite

In [1]:
import glob
import os
import json
import re
import demoji
import neologdn
import string
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertJapaneseTokenizer

In [2]:
# 乱数シードの設定
torch.manual_seed(0)

In [3]:
# モデル名を指定
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
# センテンストランスフォーマー

# トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# # モデルのロード
# model = BertModel.from_pretrained(model_name)

In [4]:
hot_reviws_path = 'reviews.csv'

In [5]:
hot_reviws = pd.read_csv(hot_reviws_path)

In [6]:
hot_reviws

,region,prefectures,salonName,salonAddress,gender,age,profession,review
0,北海道,札幌,AUBE HAIR sapporo 札幌駅前通店 【オーブ ヘアー サッポロ】,北海道札幌市中央区北１条西３－３－１１ 桂和北１条ビル ５F １号,女性,20代後半,会社員,いつもイメージ通りの髪型、髪色にしていただき本当にありがとうございます。\n今日もすごく可愛...
1,北海道,札幌,AUBE HAIR sapporo 札幌駅前通店 【オーブ ヘアー サッポロ】,北海道札幌市中央区北１条西３－３－１１ 桂和北１条ビル ５F １号,女性,20代後半,その他,今回も満足のいく仕上がりでした！！またよろしくお願いします。
2,北海道,札幌,AUBE HAIR sapporo 札幌駅前通店 【オーブ ヘアー サッポロ】,北海道札幌市中央区北１条西３－３－１１ 桂和北１条ビル ５F １号,女性,10代後半,NaN,今回もカットが上手で満足しています！\n私にとっての適度な重さで気に入ってます！\nカラーも...
3,北海道,札幌,AUBE HAIR sapporo 札幌駅前通店 【オーブ ヘアー サッポロ】,北海道札幌市中央区北１条西３－３－１１ 桂和北１条ビル ５F １号,女性,20代前半,会社員,またいつも通りのオーダーでお願いし、前髪だけそのまま伸ばすことにしました。伸びかけの前髪でも...
4,北海道,札幌,AUBE HAIR sapporo 札幌駅前通店 【オーブ ヘアー サッポロ】,北海道札幌市中央区北１条西３－３－１１ 桂和北１条ビル ５F １号,女性,40代,NaN,久しぶりに、酸性ストレートをしました。最近湿気でクセ毛がひどく乾かすのも大変でしたが、お陰で...
...,...,...,...,...,...,...,...,...
80985,北海道,札幌,髪質改善＆トリートメント MIZZO 月寒中央店,北海道札幌市豊平区月寒中央通７丁目6-20 JAビル1階,女性,40代,会社員,くせ毛でスタイリングが大変だったのですがいつも大満足の仕上がりで感謝しています。\nありがと...
80986,北海道,札幌,髪質改善＆トリートメント MIZZO 月寒中央店,北海道札幌市豊平区月寒中央通７丁目6-20 JAビル1階,女性,50代,自営業,思い通りのカットをしていただき、トリートメントもしっかりしていただき、気持ちリフレッシュでき...
80987,北海道,札幌,髪質改善＆トリートメント MIZZO 月寒中央店,北海道札幌市豊平区月寒中央通７丁目6-20 JAビル1階,女性,20代後半,会社員,自分の髪質と骨格の悩みから似合う形を考えてくださり、素敵なスタイリングでとても満足です。\n...
80988,北海道,札幌,髪質改善＆トリートメント MIZZO 月寒中央店,北海道札幌市豊平区月寒中央通７丁目6-20 JAビル1階,女性,30代前半,会社員,初ハイライトでした。\n明るすぎないさりげない感じにしたいことを伝え、\n希望通りに染めてい...


In [7]:
reviw_list = []
review_list = hot_reviws['review'].tolist()

In [ ]:
def text_preprocess(text):
  text = text.translate(str.maketrans({'\n': '', '\t': '', '\r': '', '\u3000': '', '!': '', '！': ''}))

  text = demoji.replace(string=text, repl='')

  text = neologdn.normalize(text)

  return text

In [9]:
review_list = [text_preprocess(review) for review in review_list]

In [10]:
def remove_kaomoji(text):
    # 顔文字を検出する正規表現パターン
    pattern = r'[\(\)]+[^\'\"（）\(\)]{1,10}[\(\)]+'
    
    # 正規表現を使用して顔文字を空文字列に置換
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text

In [11]:
reviews = []
reviews = [remove_kaomoji(review) for review in review_list]

In [73]:
reviews

['いつもイメージ通りの髪型、髪色にしていただき本当にありがとうございます。今日もすごく可愛く仕上げていただいて、その後のお買い物まで楽しめました。',
 '今回も満足のいく仕上がりでしたまたよろしくお願いします。',
 '今回もカットが上手で満足しています私にとっての適度な重さで気に入ってますカラーも今どきな雰囲気に仕上がったので良かったですまたよろしくお願いします♪',
 'またいつも通りのオーダーでお願いし、前髪だけそのまま伸ばすことにしました。伸びかけの前髪でも分けた時に馴染むようにカットして下さり、セットが楽ですまた、今回涼しくなるシャンプーを使っていただいたので、終わったあともしばらくスースーしてて気持ちよかったです来月またよろしくお願いします',
 '久しぶりに、酸性ストレートをしました。最近湿気でクセ毛がひどく乾かすのも大変でしたが、お陰でドライヤーも苦ではなくツルツルさらさらになり嬉しいです。また宜しくお願い致します。',
 '接客は丁寧でした、仕上がりは満足とはいきませんが、今後も来店する予定です。',
 "今回はカットとカラーをお願いしました。毛量を軽くしていただき、とっても軽くなりました。ありがとうございます。パーマも残っていて、次回キツくかけなくても大丈夫そうです(*'▽'*)次回の予約も完了です。よろしくお願いします。",
 '先日はありがとうございました。今回もまた満足のいく施術でしたまた利用したいと思います。',
 '友人の結婚式のため、アップスタイルをお願いしました。出来上がりは素敵に見えましたが、移動している間にピンが落ちてきたり、式の途中で崩れてしまいました。とても大事な友達の式だったので、アレンジが不慣れなら他のスタッフの方にもサポートいただきたかったです。すごく残念でした。',
 '前回と同じ雰囲気で素敵に仕上がりました。前髪をどうするか迷ってたのですが一旦カットして整えてから切って頂きました、臨機応変に対応してもらえて良かったです。家族からも好評で嬉しいです。',
 'いつも大満足ですありがとうございます',
 '久々の来店でした。担当のスタイリストさんは、私の髪ひつも分かって、思っている感じに仕上げて頂きました。又宜しくお願い致します。',
 '毎月定期的にカット、カラー、トリートメントをお願いしています。私からはこんな

## GiNZAによる係り受け分析 ##

In [12]:
# pip install -U ginza ja-ginza
import spacy

In [75]:
# 例
nlp = spacy.load('ja_ginza')
doc = nlp('私は選挙管理委員会です')

for sent in doc.sents:
    for token in sent:
        print(f"{token.text} <- {token.head.text} ({token.dep_})")

私 <- 選挙管理委員会 (nsubj)
は <- 私 (case)
選挙管理委員会 <- 選挙管理委員会 (ROOT)
です <- 選挙管理委員会 (cop)


In [76]:
# 日本語モデルの読み込み
nlp = spacy.load('ja_ginza')

docs = []
# 各レビューについて係り受け解析を実行
for review in reviews[:1000]:  # リストスライスで最初の20件を取得
    doc = nlp(review)
    docs.append(doc)
    print(f"レビュー: {review}")
    for sent in doc.sents:
        for token in sent:
            print(f"{token.text} <- {token.head.text} ({token.dep_})")
    print("-" * 50)  # 区切り線

レビュー: いつもイメージ通りの髪型、髪色にしていただき本当にありがとうございます。今日もすごく可愛く仕上げていただいて、その後のお買い物まで楽しめました。
いつも <- し (nsubj)
イメージ <- 通り (compound)
通り <- 髪型 (nmod)
の <- 通り (case)
髪型 <- いつも (dep)
、 <- 髪型 (punct)
髪色 <- し (obl)
に <- 髪色 (case)
し <- ござい (advcl)
て <- し (mark)
いただき <- て (fixed)
本当 <- ござい (obl)
に <- 本当 (case)
ありがとう <- ござい (discourse)
ござい <- ござい (ROOT)
ます <- ござい (aux)
。 <- ござい (punct)
今日 <- 仕上げ (advmod)
も <- 今日 (case)
すごく <- 可愛く (advcl)
可愛く <- 仕上げ (advcl)
仕上げ <- 楽しめ (advcl)
て <- 仕上げ (mark)
いただい <- て (fixed)
て <- 仕上げ (mark)
、 <- 仕上げ (punct)
その後 <- 買い物 (nmod)
の <- その後 (case)
お <- 買い物 (compound)
買い物 <- 楽しめ (obl)
まで <- 買い物 (case)
楽しめ <- 楽しめ (ROOT)
まし <- 楽しめ (aux)
た <- 楽しめ (aux)
。 <- 楽しめ (punct)
--------------------------------------------------
レビュー: 今回も満足のいく仕上がりでしたまたよろしくお願いします。
今回 <- いく (advmod)
も <- 今回 (case)
満足 <- いく (nmod)
の <- 満足 (case)
いく <- 仕上がり (acl)
仕上がり <- 願い (advcl)
でし <- 仕上がり (cop)
た <- 仕上がり (aux)
また <- 願い (advmod)
よろしく <- 願い (advmod)
お <- 願い (compound)
願い <- 願い (ROOT)
し <- 願い (aux)
ます <- 願い (aux)
。

In [85]:
index = 13
print(docs[index])
for sent in docs[index].sents:
    for token in sent:
        print(f"{token.text} <- {token.head.text} ({token.dep_})")

いつも違う方に担当していただいているのですが、前髪のサイドの髪がオーダー通りになったことがありません。今回もサイドの毛を前髪とつなげる形で耳くらいまでにしてほしいと言ったのですが、よく見たら切り残しがあって形も前髪とサイドの毛の境目が段みたいになってました。女性にとって前髪は顔のパーツの一部なので、もっと研究していただきたいと思いました。お店の落ち着く雰囲気やスタイリストさんが必要以上に話しかけてこないところが好きで今後も通いたいので、改善していただければいいなと思います。
いつ <- 担当 (nsubj)
も <- いつ (case)
違う <- 方 (acl)
方 <- 担当 (obl)
に <- 方 (case)
担当 <- なっ (advcl)
し <- 担当 (aux)
て <- 担当 (mark)
いただい <- て (fixed)
て <- 担当 (mark)
いる <- て (fixed)
の <- 担当 (mark)
です <- の (fixed)
が <- 担当 (mark)
、 <- 担当 (punct)
前髪 <- サイド (nmod)
の <- 前髪 (case)
サイド <- 髪 (nmod)
の <- サイド (case)
髪 <- なっ (nsubj)
が <- 髪 (case)
オーダー <- 通り (compound)
通り <- なっ (obl)
に <- 通り (case)
なっ <- なっ (ROOT)
た <- なっ (aux)
こと <- なっ (compound)
が <- こと (fixed)
あり <- こと (fixed)
ませ <- なっ (aux)
ん <- なっ (aux)
。 <- なっ (punct)
今回 <- 言っ (advmod)
も <- 今回 (case)
サイド <- 毛 (nmod)
の <- サイド (case)
毛 <- し (obj)
を <- 毛 (case)
前髪 <- つなげる (advmod)
と <- 前髪 (case)
つなげる <- 形 (acl)
形 <- し (obl)
で <- 形 (case)
耳 <- し (obl)
くらい <- 耳 (case)
まで <- 耳 (case)
に <- 耳 (case)
し <- 言っ (ccomp)
て <- し (mar

## KNP・eventgraphを使用した係り受け分析 ##

In [103]:
from pyknp import KNP
from pyknp_eventgraph import EventGraph

In [102]:
# Parse a document.
document = ['彼女は海外勤務が長いので、英語がうまいに違いない。', '私はそう確信していた。']
knp = KNP()
analysis = [knp.parse(sentence) for sentence in document]

# Create an EventGraph.
evg = EventGraph.build(analysis)
print(evg)  # <EventGraph, #sentences: 2, #events: 3, #relations: 1>

Exception: Can't find KNP command: knp

In [ ]:
# Extract event-to-event relations.
relations = evg.relations
print(relations)  # [<Relation, label: 原因・理由, modifier_evid: 0, head_evid: 1>]

# Take a closer look at an event-to-event relation
relation = relations[0]
print(relation.label)     # 原因・理由
print(relation.surf)      # ので
print(relation.modifier)  # <Event, evid: 0, surf: 海外勤務が長いので、>
print(relation.head)      # <Event, evid: 1, surf: 彼女は英語がうまいに違いない。>